In [66]:
!pip install rank-bm25 > None
!pip install sentence_transformers > None
!pip install transformers > None
!pip install nltk pymystem3 > None
!pip install pyenchant > None

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import pandas as pd
import nltk
import re
from tqdm import tqdm
from collections import defaultdict
import numpy as np
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import gc
from nltk import WordPunctTokenizer
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from pymystem3 import Mystem
import enchant

In [38]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Предобработка текстов

In [33]:
morph_analyzer = Mystem()
tokenizer = WordPunctTokenizer()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [69]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

In [70]:
def lemmatize(words) -> list[str]:
    lemmatized_text = morph_analyzer.lemmatize(" ".join(words))
    return [word for word in lemmatized_text if word.isalnum()]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text, language="russian")


def tokenize_by_words(text):
    return word_tokenize(text, language="russian")


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> str:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return  ' '.join(preprocessed_sentences)

In [43]:
df = pd.read_csv('/content/drive/MyDrive/resume_job_matching/Raw_Jobs.csv', delimiter=';')
df.head()

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
0,1,78577559,Главный механик,от 160000 до 160000 RUR,Более 6 лет,"Полная занятость,полный день","Вакансия компании: ООО ПК Предприятие ""ПИК"" ...",NaN,JCat.ru,Владивосток,27.03.2023,close
1,2,78693069,HTML-верстальщик,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",В креативное агентство на полный рабочий день ...,NaN,Чугунова Ирина,NaN,29.03.2023,close
2,3,78945452,Тренер (направление профессионального обучения),з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",Наши предложения: самостоятельность в...,NaN,Перекрёсток,NaN,18.04.2023,close
3,4,79352008,Старший механик на КСПГ,з/п не указана,Более 6 лет,"Полная занятость,полный день",Обязанности: _Обеспечение технически ...,NaN,Газпром гелий сервис,NaN,17.04.2023,close
4,5,79406472,System Analyst Trainee,з/п не указана,Нет опыта,"Стажировка,полный день",IT-компания Aston - компания по разработке про...,"UML,SQL,Scrum,Retail,Базы данных,Kanban,Waterf...",Aston,NaN,17.04.2023,close


In [44]:
valid_comment_pattern = r"программист|разработчик|developer"

In [45]:
df['is_valid_pattern'] = df['title'].apply(lambda x: True if not isinstance(x, float) and re.match(valid_comment_pattern, x.lower()) is not None else False)


In [46]:
df = df[df.is_valid_pattern == True]


In [47]:
#парсим описание вакансии как можем
def extract_information(job_description):
    responsibilities_pattern = r'\b(?:вам предстоит|вы будете|задачи|обязанности|чем необходимо заниматься)\b:(.*?)(?=\b(?:требования|требования к кандидатам|мы предлагаем|условия|требования:|мы ожидаем|наши пожелания|будет плюсом|будет преимуществом)\b|$)'
    requirements_pattern = r'\b(?:для нас важно|ждем от вас|мы ожидаем|наши пожелания|необходимые навыки|требования|требования к кандидатам|будет плюсом|будет преимуществом)\b:(.*?)(?=\b(?:мы предлагаем|условия|место работы|длительность рабочего дня)\b|$)'
    conditions_pattern = r'\b(?:мы предлагаем|условия)\b:(.*?)(?=\b(?:в ответ на оставленный отклик|образование)\b|$)'

    responsibilities_match = re.search(responsibilities_pattern, job_description, re.IGNORECASE | re.DOTALL)
    requirements_match = re.search(requirements_pattern, job_description, re.IGNORECASE | re.DOTALL)
    conditions_match = re.search(conditions_pattern, job_description, re.IGNORECASE | re.DOTALL)

    responsibilities = responsibilities_match.group(1).strip() if responsibilities_match else None
    requirements = requirements_match.group(1).strip() if requirements_match else None
    conditions = conditions_match.group(1).strip() if conditions_match else None

    return responsibilities, requirements, conditions

In [48]:
responsabilities, req, cond = list(), list(), list()

In [49]:
for row in tqdm(df['description']):
    responsability, requirements, conditions = extract_information(row)
    responsabilities.append(responsability)
    req.append(requirements)
    cond.append(conditions)

100%|██████████| 1219/1219 [00:00<00:00, 4182.47it/s]


In [50]:
df['responsabilities'] = responsabilities
df['requirements'] = req
df['conditions'] = cond

In [51]:
del responsabilities
del req
del cond
gc.collect()

4272

In [52]:
vacancy_text = defaultdict(str)
for idx, row in df.iterrows():
    vacancy_text[idx]  = f"Профессия: {row['title']}. Ключевые навыки: {row['key_skills']}. Обязанности: {row['responsabilities']}. Требования: {row['requirements']}. Условия {row['conditions']}"

In [53]:
df['text'] = df.index.map(vacancy_text)

In [54]:
query = """Программист C++, Web-программист, разработчик аппаратуры,.Опыт работы 5 лет 8 месяцев  Март 2019 — по настоящее время 3 месяца Удаленная работа Обнинск Web-программист Доработка сайта www.autolarek24.ru  Октябрь 2013 — по настоящее время 5 лет 8 месяцев АО «ГНЦ РФ – ФЭИ» Обнинск , www.ippe.ru Энергетика ... Атомная энергетика (генерация электроэнергии, АЭС) Инженер-программист Разработка ПО для операционных систем жесткого реального времени. Разработка драйверов, многопоточных приложений, клиент-серверных приложений. Разработка микропроцессорных систем. Работа с промышленными интерфейсами и протоколами: ModBus, CAN, RS422/485. Разработка документации ПО. Работа с измерительной аппаратурой с очень высокой точностью измерения и написание ПО для мониторинга и фильтрации измерений, а так же участие в разработке, настройке и тестировании данной аппаратуры.  Июнь 2017 — Ноябрь  2017 6 месяцев ООО "Персона" Обнинск Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Веб-разработчик Разработка веб-сайтов, концепция MVC, командная разработка (Git), Laravel. Проект www.combeep.com"""

## ВM25

In [71]:
docs = df['text'].tolist()
preprocessed_docs = [doc for doc in docs]

In [72]:
tokenized_docs = [doc.split() for doc in preprocessed_docs]
bm25 = BM25Okapi(tokenized_docs)

In [73]:
def bm25_search(query):
    scores = bm25.get_scores(query.split())
    return scores

In [74]:
# Шаг 1: BM25 поиск
bm25_scores = bm25_search(query)

In [75]:
bm25_scores

array([ 6.58140977,  8.78713466, 10.14122198, ...,  0.        ,
       12.68095199,  5.44583047])

In [76]:
def sbert_narrowing(query, candidates):
    model = SentenceTransformer('intfloat/multilingual-e5-large')
    query_embedding = model.encode(query)
    candidate_embeddings = model.encode(candidates)
    similarities = util.pytorch_cos_sim(query_embedding, candidate_embeddings)[0]
    sorted_indices = similarities.argsort().tolist()[::-1]
    narrowed_candidates = [candidates[i] for i in sorted_indices]
    return narrowed_candidates[:5]  # Выбираем топ 5 кандидатов

In [82]:
top_candidates = [doc for _, doc in sorted(zip(bm25_scores, docs), reverse=True)[:10]]
top_candidates

['Профессия: Программист станков с ЧПУ. Ключевые навыки: nan. Обязанности: 1. Написание эффективных программ для токарных и фрезерных обрабатывающих центров (минимизация машинного времени изготовления детали, износа инструмента, нагрузок на станок)   2. Проработка технологии изготовления деталей   3. Расчет времени изготовления   4. Построение 3Д моделей по чертежам   5. Разработка приспособлений. Требования: 1. Опыт написания программ   2. Желателен опыт работы оператором-наладчиком. Условия None',
 'Профессия: Программист С++/Qt (junior). Ключевые навыки: nan. Обязанности: Разработка и поддержка ПО на C++ (QT) под Windows, Linux.. Требования: профильное высшее-техническое образование      знание структур данных и алгоритмов      знание принципов объектно-ориентированного программирования (ООП)      знание принципов SOLID      знание библиотеки STL      знание библиотеки Qt      желателен опыт работы в Visual Studio 2015/2017 (С++)      желателен опыт работы в QtCreator      желателен

In [83]:
torch.cuda.empty_cache()

In [84]:
# Шаг 2: SBERT для сужения воронки
narrowed_candidates = sbert_narrowing(query, top_candidates)

In [85]:
narrowed_candidates

['Профессия: Программист-разработчик. Ключевые навыки: nan. Обязанности: Разработка программ на JS, Python. Требования: Общие требования  1) Опыт работы в компании от года либо опыт разработки сайтов на технологиях из требований  2) Готовность к работе с разными технологиями и задачами Требования Frontend  1) Знание базового JavaScript  2) Знание базового HTML  3) Знание NextJS  4) Знание React  5) Опыт работы с MobX или Redux  6) Понимание работы с REST API  7) Работа с адаптивной вёрсткой  8) Знание базового CSS, Grid, Flex  Желательно  1) Знание SCSS  2) Знание Webpack  3) Понимание основ оптимизации Веб-приложений Требования Backend  1) Знание базового JavaScript  2) Знание NodeJS  3) Знание ExpressJS  4) Работа с SQL СУБД и препроцессором Knex  5) Знание принципов построения БД и опыт работы с PostgreSQL  6) Понимание принципов и паттернов построения API сервера  Желательно  1) Понимание кластеризации и распределения нагрузки  2) Ументе работы с веб-сервером  3) Умение работы с li

## Реранкер

In [86]:
from sentence_transformers import CrossEncoder
from pprint import pprint
cross_model = CrossEncoder('PitKoro/cross-encoder-ru-msmarco-passage', max_length=2048)


def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores

model_inputs = [[query, item[:1024]] for item in narrowed_candidates]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'Text': inp, 'Score': score} for inp, score in zip(narrowed_candidates, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)


print("\n")
for result in ranked_results:
    print('\t',pprint(result))



{'Score': 0.53047097,
 'Text': 'Профессия: Программист С++/Qt (junior). Ключевые навыки: nan. '
         'Обязанности: Разработка и поддержка ПО на C++ (QT) под Windows, '
         'Linux.. Требования: профильное высшее-техническое образование      '
         'знание структур данных и алгоритмов      знание принципов '
         'объектно-ориентированного программирования (ООП)      знание '
         'принципов SOLID      знание библиотеки STL      знание библиотеки '
         'Qt      желателен опыт работы в Visual Studio 2015/2017 (С++)      '
         'желателен опыт работы в QtCreator      желателен опыт работы с '
         'TCP/IP      желателен опыт разработки многопоточных приложений.    '
         'Дополнительно приветствуется:        технический английский язык.. '
         'Условия None'}
	 None
{'Score': 0.3123988,
 'Text': 'Профессия: Разработчик ПО (ЦФТ-Банк). Ключевые навыки: nan. '
         'Обязанности: None. Требования: Опыт внедрения либо глубокого '
         'програ